In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Folder path 
path = r'/Users/crystalosuna/Documents/05-2023 Instacart Basket Analysis'

In [3]:
#import orders
df_ords = pd.read_csv (os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

In [4]:
#import products
df_prods = pd.read_csv (os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

# Step 1: Consistency Checks for df_prods

In [37]:
#check for mixed-type data in df_prods
for col in df_prods.columns.tolist():
    weird = (df_prods[[col]].applymap(type) != df_prods[[col]].iloc[0].apply(type)).any(axis= 1)
    if len (df_prods[weird]) > 0:
        print (col)

product_name


Missing Values in df_prods

In [5]:
#finding missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [6]:
df_nan = df_prods [df_prods['product_name'].isnull()==True]

In [7]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [8]:
df_prods.shape

(49693, 5)

In [9]:
#create a new dataframe
df_prods_clean = df_prods[df_prods['product_name'].isnull()== False]

In [10]:
df_prods_clean.shape

(49677, 5)

Duplicates in df_prods

In [11]:
#finding duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [12]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


Adressing duplicates in df_prods

In [13]:
df_prods_clean.shape

(49677, 5)

In [14]:
#dropping duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [15]:
df_prods_clean_no_dups.shape

(49672, 5)

Decriptive Statistics

In [18]:
#Descriptive statistics for df_prods_clean_no_dups
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


Max values for prices are too high and should be investigated.

In [19]:
#investigate max "prices"
df_prods_clean_no_dups.loc[df_prods_clean_no_dups['prices']==99999]

,product_id,product_name,aisle_id,department_id,prices
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


In [20]:
#check for products that cost more than $50
df_prods_clean_no_dups[df_prods_clean_no_dups["prices"]>50.0]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


These products prices are incorrect and should be corrected.

In [22]:
#replace 'prices' values for milkfat cottage cheese and 2% reduced fat milk
df_prods_clean_no_dups = df_prods_clean_no_dups.replace({"prices":{99999.0:9.99,14900.0:1.49}})

In [23]:
#check that changes have been made
df_prods_clean_no_dups[df_prods_clean_no_dups["product_id"]==21553]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,1.49


In [24]:
#check that changes have been made
df_prods_clean_no_dups[df_prods_clean_no_dups["product_id"]==33664]

,product_id,product_name,aisle_id,department_id,prices
33666,33664,2 % Reduced Fat Milk,84,16,9.99


# Step 2: Consistency Checks for df_ords

In [25]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [27]:
#drop unnamed column for df_ords
df_ords = df_ords.drop(columns = ['Unnamed: 0'])

In [29]:
#check if unnamed column dropped
df_ords.describe()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


# Step 3: Check for mixed-type data in df_ords

In [30]:
#check for mixed-type data in df_ords
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis= 1)
    if len (df_ords[weird]) > 0:
        print (col)
                                        

Step 4: No mixed-type data.

# Step 5: Check for missing values in df_ords

In [39]:
#check for missing values in df_ords 
df_ords.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [32]:
#create subset to show missing values in df_ords
df_ords_nan = df_ords[df_ords['days_since_prior_order'].isnull()==True]

In [33]:
df_ords_nan

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...
3420930,969311,206205,1,4,12,NaN
3420934,3189322,206206,1,3,18,NaN
3421002,2166133,206207,1,6,19,NaN
3421019,2227043,206208,1,1,15,NaN


Step 6: All missing values for days since prior order are for order number 1 which makes sense. There are no prior days because it is their first order. Will leave the data how it is.

# Step 7: Check for duplicates in df_ords

In [35]:
#Check for duplicates in df_ords
df_ords_dups = df_ords[df_ords.duplicated()]

In [36]:
df_ords_dups

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


Step 8: No duplicates found.

# EXPORT DATAFRAMES

In [40]:
#Export df_prods_clean_no_dups
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked'))

In [41]:
df_ords.to_csv(os.path.join(path,'02 Data', 'Prepared Data', 'orders_checked.csv'))